In [208]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# -----------------------------------------------------------------------------------------------
# начальные значения
v0 = 200                                # начальная скорость БЦ, м/с
theta = 30
phi = 20

geo_width = 50                          # геодезическая широта (тк нам не дано местоположение объекта)
geo_longitude = 60                      # геодезическая долгота

# -----------------------------------------------------------------------------------------------
# константы
a = 6378245                             # большая полуось Земли
b = 6356863                             # малая полуось Земли

GM = 3.986004418e14                     # произведение гравитационной постоянной Земли на массу
c_0 = 2.202095e10                       # коэффициент гравитационного поля Земли
omega = 7.292115e-5                     # угловая скорость вращения Земли
f = 298.3                               # коэффициент сжатия Земли
e_squared = (2*f-1)/((f - 1)**2)        # квадрат эксцентриситета Земли

# -----------------------------------------------------------------------------------------------
# формулы для пересчета из ГПСК в МЗСК
r = a/np.sqrt(1+(1/(1-1/f)**2 - 1)*(np.sin(geo_width))**2)                # расстояние от начала ГПСК до начала МЗСК
H = 5                                                                     # высота объекта над поверхностью эллипсоида

sin_geo_width = np.sin(geo_width)                                         
cos_geo_width = np.cos(geo_width)
sin_geo_longitude = np.sin(geo_longitude)
cos_geo_longitude = np.cos(geo_longitude)

# матрица направляющих косинусов
A_mg = np.array([[-cos_geo_longitude*sin_geo_width, -sin_geo_longitude*sin_geo_width, cos_geo_width], [cos_geo_longitude*cos_geo_width, sin_geo_longitude*cos_geo_width, sin_geo_width], [-sin_geo_longitude, cos_geo_longitude, 0]])

# геоцентрическая широта начала координат МЗСК
geocentric_latitude = np.arctan((1-1/f)**2 * np.tan(geo_width))

sin_geocentric_latitude = np.sin(geocentric_latitude)
cos_geocentric_latitude = np.cos(geocentric_latitude)

# координаты центра МЗСК в ГПСК
x_0 = np.fabs(np.array([r*cos_geocentric_latitude*cos_geo_longitude + H*cos_geo_width*cos_geo_longitude, r*cos_geocentric_latitude*sin_geo_longitude + H*cos_geo_width*cos_geo_longitude, r*sin_geocentric_latitude + H*sin_geo_width]))
# -----------------------------------------------------------------------------------------------

In [209]:
# -----------------------------------------------------------------------------------------------
# Функция, позволяющая получить первые и вторые производные координат
# -----------------------------------------------------------------------------------------------
def equations(coordinate, speed):
    x, y, z = coordinate
    vx, vy, vz = speed
    
    # вычисление первых производных 
    dx_dt = vx
    dy_dt = vy
    dz_dt = vz
    
    # учет ускорения свободного падения g_0
    r_0 = np.sqrt(x**2 + y**2 + z**2)
    A = GM/r_0**3
    B = (3*c_0)/r_0**2
    C = (15*c_0)/r_0**4
    D = (6*c_0)/r_0**2
    
    dvx_dt = -A*x + A*B*x - A*C*x*z**2
    dvy_dt = -A*y + A*B*y - A*C*y*z**2
    dvz_dt = -A*z + A*B*z + A*D*z - A*C*z**3
    
    # учет центростремительного ускорения БЦ
    dvx_dt += omega**2 * x
    dvy_dt += omega**2 * y
    dvz_dt += 0
    
    # учет кориолисова ускорения
    dvx_dt += 2 * omega * dy_dt
    dvy_dt += -2 * omega * dx_dt
    dvz_dt += 0
    
    # учет аэродинамического ускорения 
    v = np.sqrt(vx**2 + vy**2 + vz**2)
    # тк в задании не даны начальная масса БЦ, и другие данные, чтобы посчитать баллистический коэффициент, я приму gamma = 0,347 (нашла значения для пули)
    gamma = 0.347 # для более точных расчетов нужно в процессе полета пересчитывать это значение (но тк начальных параметров нет, здесь я этого не делаю)
    
    s = z/r_0                                     # синус широты цели
    h = r_0 - a/(np.sqrt(1 + e_squared * s**2))   # высота БЦ над поверхностью Земли
    rho = 10**(-1.8977 * 10**-29 * h**6 + 8.7419 * 10**-24 * h**5 - 1.4909 * 10**-18 * h**4 + 1.1591 * 10**(-13) * h**3 - 4.0740 * 10**(-9) * h**2 - 8.0463 * 10**(-6) * h + 2.0667 * 10**(-2))
    E = (-gamma * rho * v)/2
    
    dvx_dt += E * dx_dt
    dvy_dt += E * dy_dt
    dvz_dt += E * dz_dt
    
    return np.array([dx_dt, dy_dt, dz_dt, dvx_dt, dvy_dt, dvz_dt])

In [210]:
# -------------------------------------------------------------------------------------
# Метод Рунге-Кутты 4 порядка
# -------------------------------------------------------------------------------------
def runge_kutta_4(t0, t_end, dt, initial_coordinate, initial_speed):
    t = t0
    coordinates = [initial_coordinate]
    speeds = [initial_speed]
    
    while t < t_end:
        coordinate = coordinates[-1]
        speed = speeds[-1]
        
        
        k1 = equations(coordinate, speed)
        q1 = speed
        k2 = equations(coordinate + dt/2 * q1, speed + dt/2 * k1[-3:])
        q2 = q1 + k1[-3:] * dt/2
        k3 = equations(coordinate + dt/2 * q2, speed + dt/2 * k2[-3:])
        q3 = q1 + k2[-3:] * dt/2
        k4 = equations(coordinate + dt * q3, speed + dt * k3[-3:])
        q4 = q1 + k3[-3:] * dt                             
        
        new_coordinate = coordinate + dt/6 * (q1 + 2*q2 + 2*q3 + q4)
        coordinates.append(new_coordinate)
        
        new_speed = speed + dt/6 * (k1[-3:] + 2*k2[-3:] + 2*k3[-3:] + k4[-3:])
        speeds.append(new_speed)

        t += dt
        
    return [np.array(coordinates), np.array(speeds)]

In [211]:
# -------------------------------------------------------------------------------------
# Основная функция для получения координат и их первых производных
# -------------------------------------------------------------------------------------
def get_result():
    t0 = 0
    t_end = 10
    dt = 1
    
    initial_coordinate = np.array([1, 1, 1])
    initial_speed = np.array([np.fabs(v0 * np.sin(theta) * np.cos(phi)), np.fabs(v0 * np.sin(theta) * np.sin(phi)), np.fabs(v0 * np.cos(theta))])

    result = runge_kutta_4(t0, t_end, dt, initial_coordinate, initial_speed)
    
    coordinate = result[0]
    speed = result[1]

    # извлечение координат и времени из результата
    x = coordinate[:, 0]
    y = coordinate[:, 1]
    z = coordinate[:, 2]
    vx = speed[:, 0]
    vy = speed[:, 1]
    vz = speed[:, 2]
    t = np.arange(t0, t_end + dt, dt)
    t_list = t.tolist()

    return [x, y, z, vx, vy, vz, t_list] 

In [212]:
# -------------------------------------------------------------------------------------
# Выделение из ответа координат и скоростей для последующих расчетов
# -------------------------------------------------------------------------------------
result = get_result()
x_g = result[0]
y_g = result[1]
z_g = result[2]
vx = result[3]
vy = result[4]
vz = result[5]
t = result[6]
# -------------------------------------------------------------------------------------
# Вычисление высоты, скорости и ускорения
# -------------------------------------------------------------------------------------
# v = np.sqrt(vx**2 + vy**2 + vz**2)
# r_0 = np.sqrt(x**2 + y**2 + z**2)
# s = z/r_0                                     # синус широты цели
# result_data['r_0'] = list(map(int, r_0))
# h = r_0 + a/(np.sqrt(1 + e_squared * s**2)) - r  # высота БЦ над поверхностью Земли


# L = np.sqrt(x**2 + y**2)
# -------------------------------------------------------------------------------------

In [213]:
# Перевод координат из ГПСК в МЗСК
# def recalculation_gpsk_to_mhsk()

# i = 0
# while (i < 11):
#     coordinates_g = np.array([x_g[i], y_g[i], z_g[i]])
#     print(coordinates_g)
#     coordinates_m = np.dot(A_mg, coordinates_g - x_0)
#     print(coordinates_m)
#     i += 1
    

In [214]:
# -------------------------------------------------------------------------------------
# Визуализация высоты от времени
# -------------------------------------------------------------------------------------
# plt.figure()
# plt.plot(t, h)
# plt.xlabel('Время t')
# plt.ylabel('Высота h')
# plt.title('Высота от времени h(t)')
# plt.grid(True)

plt.show()


In [215]:
# -------------------------------------------------------------------------------------
# Визуализация высоты от дальности
# -------------------------------------------------------------------------------------
# plt.figure()
# plt.plot(L, h)
# plt.xlabel('Дальность L')
# plt.ylabel('Высота h')
# plt.title('Высота от дальности L(h)')
# plt.grid(True)

plt.show()

In [216]:
# -------------------------------------------------------------------------------------
# Визуализация скорости от времени
# -------------------------------------------------------------------------------------
# plt.figure()
# plt.plot(t, v)
# plt.xlabel('Время t')
# plt.ylabel('Скорость v')
# plt.title('Скорость от времени v(t)')
# plt.grid(True)

plt.show()

In [217]:
# -------------------------------------------------------------------------------------
# Визуализация скорости от дальности
# -------------------------------------------------------------------------------------
# plt.figure()
# plt.plot(L, v)
# plt.xlabel('Дальность L')
# plt.ylabel('Скорость v')
# plt.title('Скорость от дальности v(L)')
# plt.grid(True)

plt.show()

In [218]:
# -------------------------------------------------------------------------------------
# Визуализация ускорения от времени
# -------------------------------------------------------------------------------------
# plt.figure()
# plt.plot(t, a)
# plt.xlabel('Время')
# plt.ylabel('Ускорение')
# plt.title('Ускорение от времени')
# plt.grid(True)

plt.show()

In [219]:
# -------------------------------------------------------------------------------------
# Визуализация ускорения от дальности
# -------------------------------------------------------------------------------------
# plt.figure()
# plt.plot(x, a)
# plt.xlabel('Дальность')
# plt.ylabel('Ускорение')
# plt.title('Ускорение от дальности')
# plt.grid(True)

plt.show()

In [220]:
# -------------------------------------------------------------------------------------
# Визуализация траектории ракеты в 3D
# -------------------------------------------------------------------------------------
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot(x, y, z)
# ax.set_xlabel('x')
# ax.set_ylabel('y')
# ax.set_zlabel('z')
# ax.set_title('Траектория баллистической ракеты')

plt.show()

In [221]:
# -------------------------------------------------------------------------------------
# Составление таблицы с основными данными
# -------------------------------------------------------------------------------------
result_data = pd.DataFrame()
result_data['t'] = list(map(int, result[6]))
result_data['x'] = list(map(int, result[0]))
result_data['y'] = list(map(int, result[1]))
result_data['z'] = list(map(int, result[2]))
# result_data['h'] = list(map(int, h))

print("Все расчеты проводятся в системе ГПСК(Гринвичская прямоугольная система координат). Начало ГПСК находится в центре Земли.")    
result_data

Все расчеты проводятся в системе ГПСК(Гринвичская прямоугольная система координат). Начало ГПСК находится в центре Земли.


,t,x,y,z
0,0,1,1,1
1,1,-187707258814593058734080,-187679867463083216601088,375387193967926610755584
2,2,-375441873887838248894464,-375332326176096094519296,750774407167857913757696
3,3,-563203856627165856530432,-562957405241154258075648,1126161620367789149650944
4,4,-750993204037670053871616,-750555101664808041185280,1501548833567720385544192
5,5,-938809913124153794232320,-938125412453898828906496,1876936046767651621437440
6,6,-1126653980891129248219136,-1125668334615559392985088,2252323259967582857330688
7,7,-1314525404342817266860032,-1313183865157213556310016,2627710473167514093223936
8,8,-1502424180483148254019584,-1500672001086576931110912,3003097686367445329117184
9,9,-1690350306315761226874880,-1688132739411655576780800,3378484899567376565010432
